In [ ]:
import requests
import pandas as pd
import os

# --- CONFIGURACIÓN ---
MI_TICKET = "CAMBIAR_POR_TU_KEY_AQUI"
NOMBRE_ARCHIVO_OC = "ordenes_de_compra_historico.csv"

def recolectar_y_actualizar_oc():
    """
    Obtiene las Órdenes de Compra (OC) recientes, las combina con el
    histórico existente, elimina duplicados y guarda el resultado.
    """
    df_historico_oc = pd.DataFrame()

    if os.path.exists(NOMBRE_ARCHIVO_OC):
        print(f"Encontrado archivo histórico '{NOMBRE_ARCHIVO_OC}'. Cargando datos...")
        df_historico_oc = pd.read_csv(NOMBRE_ARCHIVO_OC)
        print(f"Se cargaron {len(df_historico_oc)} Órdenes de Compra históricas.")
    else:
        print(f"Archivo '{NOMBRE_ARCHIVO_OC}' no encontrado. Se creará uno nuevo.")

    print("Consultando la API para obtener las últimas Órdenes de Compra...")
    url_api = f"https://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?ticket={MI_TICKET}"
    
    try:
        response = requests.get(url_api, timeout=60)
        if response.status_code == 200:
            datos_json = response.json()
            listado_oc = datos_json.get('Listado', [])
            
            if not listado_oc:
                print("API no retornó nuevas Órdenes de Compra en esta consulta. Finalizando.")
                return

            df_nuevos = pd.DataFrame(listado_oc)
            print(f"Se obtuvieron {len(df_nuevos)} registros de la API.")

            df_consolidado = pd.concat([df_historico_oc, df_nuevos], ignore_index=True)
            
            registros_antes = len(df_consolidado)
            
            # === CORRECCIÓN APLICADA AQUÍ ===
            # Se usa 'Codigo' como el identificador único para eliminar duplicados.
            df_consolidado.drop_duplicates(subset=['Codigo'], keep='last', inplace=True)
            
            registros_despues = len(df_consolidado)
            nuevos_registros = registros_despues - len(df_historico_oc)
            
            print(f"Proceso de consolidación: {registros_antes} registros -> {registros_despues} registros únicos.")

            df_consolidado.to_csv(NOMBRE_ARCHIVO_OC, index=False, encoding='utf-8-sig')
            print(f"\n✅ ¡Proceso completado! Archivo '{NOMBRE_ARCHIVO_OC}' actualizado.")
            if nuevos_registros > 0:
                 print(f"Se agregaron {nuevos_registros} Órdenes de Compra nuevas al histórico.")
            else:
                 print("No se agregaron Órdenes de Compra nuevas en esta ejecución.")

        else:
            print(f"❌ Error al consultar la API. Código de estado: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Ocurrió un error de conexión: {e}")

# --- Ejecutar la función ---
if __name__ == "__main__":
    recolectar_y_actualizar_oc()

Archivo 'ordenes_de_compra_historico.csv' no encontrado. Se creará uno nuevo.
Consultando la API para obtener las últimas Órdenes de Compra...
Se obtuvieron 16797 registros de la API.
Proceso de consolidación: 16797 registros -> 16797 registros únicos.

✅ ¡Proceso completado! Archivo 'ordenes_de_compra_historico.csv' actualizado.
Se agregaron 16797 Órdenes de Compra nuevas al histórico.
